# PyTorch: Creating Datasets from AIS Backend
In the rapidly evolving field of machine learning, efficient data handling is crucial for training models effectively. This guide explores how to leverage AIStore (AIS), a scalable object storage solution, to create and manage datasets directly within PyTorch. We'll cover the integration of AIS with PyTorch through two custom dataset classes: AISDataset for map-style datasets and AISIterDataset for iterable datasets. These classes facilitate seamless access to data stored in AIS, supporting a variety of machine learning workflows. For details refer to [README](https://github.com/NVIDIA/aistore/tree/main/python/aistore/pytorch).


In [3]:
# Imports
import os
import torch
from torch.utils.data import DataLoader
from aistore.pytorch.dataset import AISDataset, AISIterDataset
from aistore.sdk import Client

## Setup client and necessary buckets 

In [4]:
ais_url = os.getenv("AIS_ENDPOINT", "http://localhost:8080")
client = Client(ais_url)
bucket = client.bucket("my-bck").create(exist_ok=True)

### Create some objects in the bucket

In [7]:
object_names = [f"example_obj_{i}" for i in range(10)]
for name in object_names:
    bucket.object(name).put_content(f"{name} - object content".encode("utf-8"))

### Creating a Map-Style Dataset

In [ ]:
map_dataset = AISDataset(client_url=ais_url, urls_list='ais://my-bck')

for i in range(len(map_dataset)): # calculate length of all items present using len() function
    print(map_dataset[i]) # get object url and byte array of the object

# Create a DataLoader from the dataset
map_data_loader = DataLoader(map_dataset, batch_size=10, num_workers=2)


### Creating a Iterable-Style Dataset

In [ ]:
iter_dataset = AISIterDataset(client_url=ais_url, ais_source_list=bucket)
for sample in iter_dataset:
    print(sample) # get object url and byte array of the object

# Create a DataLoader from the dataset
iter_data_loader = DataLoader(iter_dataset, batch_size=10, num_workers=2)

**Note:** We can also provide an etl_name (which is present in our cluster) to the Dataset to apply an etl to the objects. For example -  `AISDataset(client_url=ais_url, urls_list='ais://my-bck', etl_name=your_etl_name)`